In [29]:
import csv
import io
import os
import time
import uuid # <-- NEW: Import uuid
from langchain_chroma import Chroma
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_docling import DoclingLoader
from langchain_docling.loader import ExportType
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter

In [30]:
# 1. Define your data
CSV_DATA = """Domain,Service,PDF_URL
Analytics,sagemaker,https://docs.aws.amazon.com/pdfs/next-generation-sagemaker/latest/userguide/next-generation-sagemaker-ug.pdf
"""

In [34]:
# 2. Define constants
CHROMA_DB_PATH = "./chroma_db_AWSDocs"
EMBEDDING_MODEL_NAME = "nomic-embed-text" 
BATCH_SIZE = 50  
CHUNK_SIZE = 1000 
CHUNK_OVERLAP = 200

In [35]:
def parse_csv_data(csv_data):
    """Parses the in-memory CSV string into a list of dictionaries."""
    service_docs = []
    f = io.StringIO(csv_data)
    reader = csv.DictReader(f)
    for row in reader:
        service_docs.append(row)
    return service_docs

In [36]:
print("Initializing Ollama embeddings...")
try:
    embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL_NAME)
    embeddings.embed_query("Test embedding")
except Exception as e:
    print(f"Error connecting to Ollama. Is it running?")

2025-11-11 19:56:01,864 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


Initializing Ollama embeddings...


In [37]:
vector_store = Chroma(
    collection_name="AWSDocs",
    persist_directory=CHROMA_DB_PATH,
    embedding_function=embeddings
)

In [38]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)

In [39]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)

In [40]:
loader = DoclingLoader(
    file_path='https://docs.aws.amazon.com/pdfs/next-generation-sagemaker/latest/userguide/next-generation-sagemaker-ug.pdf',
    export_type=ExportType.MARKDOWN,
)

In [41]:
docs_as_markdown = loader.load()

2025-11-11 19:56:11,972 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-11-11 19:56:12,053 - INFO - Going to convert document batch...
2025-11-11 19:56:12,054 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 44ae89a68fc272bc7889292e9b5a1bad
2025-11-11 19:56:12,059 - INFO - Auto OCR model selected ocrmac.
2025-11-11 19:56:12,062 - INFO - Accelerator device: 'mps'
2025-11-11 19:56:17,029 - INFO - Accelerator device: 'mps'
2025-11-11 19:56:18,364 - INFO - Processing document next-generation-sagemaker-ug.pdf
2025-11-11 19:56:42,120 - INFO - Finished converting document next-generation-sagemaker-ug.pdf in 30.61 sec.


In [42]:
docs_as_markdown

[Document(metadata={'source': 'https://docs.aws.amazon.com/pdfs/next-generation-sagemaker/latest/userguide/next-generation-sagemaker-ug.pdf'}, page_content='aws\n\n## User Guide\n\n## Amazon SageMaker\n\nC\n\n&lt;/&gt;\n\n## Amazon SageMaker: User Guide\n\nCopyright © 2025 Amazon Web Services, Inc. and/or its affiliates. All rights reserved.\n\nAmazon\'s trademarks and trade dress may not be used in connection with any product or service that is not Amazon\'s, in any manner that is likely to cause confusion among customers, or in any manner that disparages or discredits Amazon. All other trademarks not owned by Amazon are the property of their respective owners, who may or may not be affiliated with, connected to, or sponsored by Amazon.\n\n## Table of Contents\n\n| What is Amazon SageMaker? ..........................................................................................................                                                                                           

In [43]:
markdown_content = docs_as_markdown[0].page_content

In [44]:
semantic_chunks = markdown_splitter.split_text(markdown_content)


In [45]:
for chunk in semantic_chunks:
    chunk.metadata["domain"] = 'domain'
    chunk.metadata["service"] = 'service_name'
    chunk.metadata["source"] = 'url'
final_chunks = text_splitter.split_documents(semantic_chunks)
if final_chunks:
    print(f"Found {len(final_chunks)} final chunks to add.")    

Found 195 final chunks to add.


In [46]:
for chunk in final_chunks:
    print(chunk.page_content)
    print(chunk.metadata)
    print("-"*100)

aws
{'domain': 'domain', 'service': 'service_name', 'source': 'url'}
----------------------------------------------------------------------------------------------------
C  
&lt;/&gt;
{'Header 2': 'Amazon SageMaker', 'domain': 'domain', 'service': 'service_name', 'source': 'url'}
----------------------------------------------------------------------------------------------------
Copyright © 2025 Amazon Web Services, Inc. and/or its affiliates. All rights reserved.  
Amazon's trademarks and trade dress may not be used in connection with any product or service that is not Amazon's, in any manner that is likely to cause confusion among customers, or in any manner that disparages or discredits Amazon. All other trademarks not owned by Amazon are the property of their respective owners, who may or may not be affiliated with, connected to, or sponsored by Amazon.
{'Header 2': 'Amazon SageMaker: User Guide', 'domain': 'domain', 'service': 'service_name', 'source': 'url'}
---------------------

In [47]:
process_docs = final_chunks[20:]

In [53]:
chunk_ids = [str(uuid.uuid4()) for _ in final_chunks]

In [49]:
final_chunks = process_docs

In [51]:
final_chunks = final_chunks[20:]

In [52]:
final_chunks

[Document(metadata={'Header 2': 'Generative AI application development', 'domain': 'domain', 'service': 'service_name', 'source': 'url'}, page_content="Access Amazon Bedrock's capabilities through SageMaker Unified Studio to quickly build and customize your generative AI applications. This intuitive interface lets you work with highperforming foundation models (FMs) from leading companies like Anthropic, Mistral, Meta, and Amazon, and use advanced features like Amazon Bedrock Knowledge Bases, Amazon Bedrock Guardrails, Amazon Bedrock Agents, and Amazon Bedrock Flows. You can develop generative AI applications faster within SageMaker Unified Studio's secure environment, ensuring alignment with your requirements and responsible AI guidelines."),
 Document(metadata={'Header 2': 'Note', 'domain': 'domain', 'service': 'service_name', 'source': 'url'}, page_content="When should I use Bedrock in SageMaker Unified Studio versus the standalone Amazon Bedrock service?  \nAmazon Bedrock's capabil

In [54]:
for i in range(0, len(final_chunks)):
    batch_docs = final_chunks[i:i + 1]
    batch_ids = chunk_ids[i:i + 1]
    
    batch_num = (i) + 1
    total_batches = len(final_chunks) + 1

    print(f"  Adding batch {batch_num}/{total_batches}...")
    
    print(batch_docs)
    # 3. Pass both documents and ids to the vector store
    vector_store.add_documents(
        documents=batch_docs,
        ids=batch_ids
    )
    
    time.sleep(2)

  Adding batch 1/156...
[Document(metadata={'Header 2': 'Generative AI application development', 'domain': 'domain', 'service': 'service_name', 'source': 'url'}, page_content="Access Amazon Bedrock's capabilities through SageMaker Unified Studio to quickly build and customize your generative AI applications. This intuitive interface lets you work with highperforming foundation models (FMs) from leading companies like Anthropic, Mistral, Meta, and Amazon, and use advanced features like Amazon Bedrock Knowledge Bases, Amazon Bedrock Guardrails, Amazon Bedrock Agents, and Amazon Bedrock Flows. You can develop generative AI applications faster within SageMaker Unified Studio's secure environment, ensuring alignment with your requirements and responsible AI guidelines.")]


2025-11-11 20:02:52,608 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-11 20:02:54,901 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


  Adding batch 2/156...
[Document(metadata={'Header 2': 'Note', 'domain': 'domain', 'service': 'service_name', 'source': 'url'}, page_content="When should I use Bedrock in SageMaker Unified Studio versus the standalone Amazon Bedrock service?  \nAmazon Bedrock's capabilities in Amazon SageMaker Unified Studio are ideal for enterprise teams who need a governed low-code/no-code environment for collaboratively building and deploying generative AI applications, alongside unified analytics and machine learning capabilities.  \nCustomers can use the standalone Bedrock service from the AWS Management Console or Bedrock API when they want to leverage the full feature set of Bedrock including the latest agents, flow and guardrail enhancements, and the Bedrock SDK.")]


2025-11-11 20:02:56,976 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


  Adding batch 3/156...
[Document(metadata={'Header 2': 'Get started with Amazon SageMaker', 'domain': 'domain', 'service': 'service_name', 'source': 'url'}, page_content='You can view demos of Amazon SageMaker and get started by setting up a domain and project.')]


...

  Adding batch 153/156...
[Document(metadata={'Header 2': 'Note', 'domain': 'domain', 'service': 'service_name', 'source': 'url'}, page_content='Some models require acceptance of an end-user license agreement (EULA). If this applies to the model that you choose to fine-tune, Amazon SageMaker Unified Studio prompts you with a window that contains the EULA content. You are responsible for reviewing and complying with any applicable license terms and making sure they are acceptable for your use case before using the model.  \nAmazon SageMaker Unified Studio shows a page with details about the training job. Here, you can observe the status of the job as it executes.  \nThe training job might take a long time to complete. You can view it at any time from the Training jobs page.  \nWhen the training job completes, the status becomes Completed . After the job completes, you can choose Deploy to deploy the fine-tuned model to an inference endpoint.')]


2025-11-11 20:08:14,618 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


  Adding batch 154/156...
[Document(metadata={'Header 2': 'API Documentation for Amazon SageMaker', 'domain': 'domain', 'service': 'service_name', 'source': 'url'}, page_content='You can use the following guides to work with Amazon SageMaker programmatically:  \n- SQL analytics\n- Amazon Redshift API reference\n- Data processing\n- Amazon EMR API reference\n- AWS Glue API reference\n- Amazon Athena API reference\n- Model development\n- Amazon SageMaker AI API reference\n- Gen AI app development\n- Amazon Bedrock API reference\n- Data and AI Governance\n- Amazon DataZone API Reference\n- SageMaker lakehouse architecture\n- Lake Formation API reference\n- Catalog objects API reference')]


2025-11-11 20:08:16,713 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


  Adding batch 155/156...
[Document(metadata={'Header 2': 'Document history for the Amazon SageMaker User Guide', 'domain': 'domain', 'service': 'service_name', 'source': 'url'}, page_content='The following table describes the documentation releases for Amazon SageMaker.  \n| Change          | Description                   | Date          |\n|-----------------|-------------------------------|---------------|\n| Initial release | Initial release of the Amazon | June 13, 2025 |')]


In [27]:
import csv
import io
import os
import time
import uuid # <-- NEW: Import uuid
from langchain_chroma import Chroma
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_docling import DoclingLoader
from langchain_docling.loader import ExportType
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter

# 1. Define your data
CSV_DATA = """Domain,Service,PDF_URL
Analytics,sagemaker,https://docs.aws.amazon.com/pdfs/next-generation-sagemaker/latest/userguide/next-generation-sagemaker-ug.pdf
"""

# 2. Define constants
CHROMA_DB_PATH = "./chroma_db_cloudyintel"
EMBEDDING_MODEL_NAME = "nomic-embed-text" 
BATCH_SIZE = 1  
CHUNK_SIZE = 2000 # Safety net: Max characters per *final* chunk
CHUNK_OVERLAP = 200

def parse_csv_data(csv_data):
    """Parses the in-memory CSV string into a list of dictionaries."""
    service_docs = []
    f = io.StringIO(csv_data)
    reader = csv.DictReader(f)
    for row in reader:
        service_docs.append(row)
    return service_docs

def load_and_process_pdfs(service_docs):
    """
    Downloads, loads, splits, and embeds all PDFs using Docling,
    then saves them to Chroma.
    """
    
    # 3. Initialize components
    
    # Embedding model (Using Ollama)
    print("Initializing Ollama embeddings...")
    try:
        embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL_NAME)
        embeddings.embed_query("Test embedding")
    except Exception as e:
        print(f"Error connecting to Ollama. Is it running?")
        return

    # Chroma vector store
    vector_store = Chroma(
        persist_directory=CHROMA_DB_PATH,
        embedding_function=embeddings
    )
    
    # Safety net splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP
    )
    
    print(f"Initialized Chroma DB at {CHROMA_DB_PATH}")
    
    # 4. Process each service document
    total_services = len(service_docs)
    for i, service in enumerate(service_docs):
        domain = service['Domain']
        service_name = service['Service']
        url = service['PDF_URL']
        
        print(f"\n--- Processing {i+1}/{total_services}: {domain} - {service_name} ---")
        print(f"URL: {url}")
        
        try:
            start_time = time.time()
            
            # A. Load PDF using Docling's default AI chunker
            print("Initializing DoclingLoader (using default AI chunking)...")
            loader = DoclingLoader(
                file_path=url,
                # NO export_type specified, so it uses default ExportType.DOC_CHUNKS
            )
            
            print("Loading and parsing with Docling (this may take a while)...")
            # 'docs' is now a list of AI-powered semantic chunks
            docs = loader.load()
            
            if not docs:
                print("Docling returned no content. Skipping.")
                continue
            
            # B. Add our custom metadata to each semantic chunk
            for doc in docs:
                doc.metadata["domain"] = domain
                doc.metadata["service"] = service_name
                doc.metadata["source"] = url

            # C. Stage 2 Split: Safety net
            print(f"Applying safety split to {len(docs)} AI chunks...")
            final_chunks = text_splitter.split_documents(docs)
            
            # D. Embed and add final chunks to Chroma
            if final_chunks:
                print(f"Found {len(final_chunks)} final chunks to add.")
                
                # Create a list of unique IDs
                chunk_ids = [str(uuid.uuid4()) for _ in final_chunks]
                
                # Batch both the documents and the IDs
                for i in range(0, len(final_chunks), BATCH_SIZE):
                    batch_docs = final_chunks[i:i + BATCH_SIZE]
                    batch_ids = chunk_ids[i:i + BATCH_SIZE]
                    
                    batch_num = (i // BATCH_SIZE) + 1
                    total_batches = (len(final_chunks) // BATCH_SIZE) + 1

                    print(f"  Adding batch {batch_num}/{total_batches}...")
                    
                    vector_store.add_documents(
                        documents=batch_docs,
                        ids=batch_ids
                    )
                
                end_time = time.time()
                print(f"Successfully processed {service_name} in {end_time - start_time:.2f} seconds.")
            else:
                print(f"No text chunks extracted from {service_name}.")

        except Exception as e:
            print(f"Error: Failed to process {service_name} from {url}. Skipping. Details: {e}")

    print("\n--- All documents processed! ---")
    print(f"Vector database is persistent and saved in '{CHROMA_DB_PATH}'")
    
    return vector_store


In [24]:
OllamaEmbeddings(model=EMBEDDING_MODEL_NAME).embed_query("Initial connection test")
print("Ollama connection successful.")

2025-11-11 19:52:47,981 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


Ollama connection successful.


In [ ]:
if os.path.exists(CHROMA_DB_PATH):
    print(f"Database already exists at {CHROMA_DB_PATH}.")
    print("To re-build, please delete this directory and run again.")
else:


Database already exists at ./chroma_db_AWSDocs.
To re-build, please delete this directory and run again.


In [28]:
service_docs = parse_csv_data(CSV_DATA)
load_and_process_pdfs(service_docs)

Initializing Ollama embeddings...


2025-11-11 19:53:43,159 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-11 19:53:43,182 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


Initialized Chroma DB at ./chroma_db_cloudyintel

--- Processing 1/1: Analytics - sagemaker ---
URL: https://docs.aws.amazon.com/pdfs/next-generation-sagemaker/latest/userguide/next-generation-sagemaker-ug.pdf
Initializing DoclingLoader (using default AI chunking)...
Loading and parsing with Docling (this may take a while)...


2025-11-11 19:53:45,772 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-11-11 19:53:45,834 - INFO - Going to convert document batch...
2025-11-11 19:53:45,837 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 44ae89a68fc272bc7889292e9b5a1bad
2025-11-11 19:53:45,847 - INFO - Auto OCR model selected ocrmac.
2025-11-11 19:53:45,849 - INFO - Accelerator device: 'mps'
2025-11-11 19:53:50,863 - INFO - Accelerator device: 'mps'
2025-11-11 19:53:52,338 - INFO - Processing document next-generation-sagemaker-ug.pdf
2025-11-11 19:54:28,595 - INFO - Finished converting document next-generation-sagemaker-ug.pdf in 44.15 sec.
Token indices sequence length is longer than the specified maximum sequence length for this model (3596 > 512). Running this sequence through the model will result in indexing errors


Applying safety split to 203 AI chunks...
Found 203 final chunks to add.
  Adding batch 1/204...


2025-11-11 19:54:29,815 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


Error: Failed to process sagemaker from https://docs.aws.amazon.com/pdfs/next-generation-sagemaker/latest/userguide/next-generation-sagemaker-ug.pdf. Skipping. Details: Expected metadata value to be a str, int, float, bool, SparseVector, or None, got {'schema_name': 'docling_core.transforms.chunker.DocMeta', 'version': '1.0.0', 'doc_items': [{'self_ref': '#/texts/0', 'parent': {'$ref': '#/body'}, 'children': [], 'content_layer': 'body', 'label': 'text', 'prov': [{'page_no': 1, 'bbox': {'l': 9.27027010472973, 't': 782.0892856775794, 'r': 39.06756740202703, 'b': 771.5178571061508, 'coord_origin': 'BOTTOMLEFT'}, 'charspan': [0, 3]}]}], 'origin': {'mimetype': 'application/pdf', 'binary_hash': 16562198080799201207, 'filename': 'next-generation-sagemaker-ug.pdf'}} which is a dict in upsert.

Try filtering complex metadata from the document using langchain_community.vectorstores.utils.filter_complex_metadata.

--- All documents processed! ---
Vector database is persistent and saved in './chro

In [ ]:

def main():
    print("Checking Ollama connection...")
    try:
        OllamaEmbeddings(model=EMBEDDING_MODEL_NAME).embed_query("Initial connection test")
        print("Ollama connection successful.")
    except Exception as e:
        print("Error: Could not connect to Ollama.")
        print("Please make sure the Ollama application is running and you have run:")
        print(f"ollama pull {EMBEDDING_MODEL_NAME}")
        return

    if os.path.exists(CHROMA_DB_PATH):
        print(f"Database already exists at {CHROMA_DB_PATH}.")
        print("To re-build, please delete this directory and run again.")
    else:
        service_docs = parse_csv_data(CSV_DATA)
        load_and_process_pdfs(service_docs)

if __name__ == "__main__":
    main()